# Procesamiento de Encuesta Congresistas

In [26]:
## Importar librerías
import pandas as pd
import numpy as np
import re
import openpyxl

In [54]:
#  Cargar datos

archivo = '/Users/user/projects/projects/zentropyETL/data/results-survey798946.xlsx'


df = pd.read_excel(archivo)

# Explorar los datos iniciales
print('Información inicial del dataset:')
print(f'Dimensiones: {df.shape}')

# Eliminar espacios
df.columns = (
    df.columns
    .str.strip()                          
    .str.replace(r'\s+', ' ', regex=True) 
)


# Ver todas las columnas
print('Columnas del dataset:')
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. '{col}'")


Información inicial del dataset:
Dimensiones: (47, 137)
Columnas del dataset:
  1. 'ID de respuesta'
  2. 'Fecha de envío'
  3. 'Última página'
  4. 'Lenguaje inicial'
  5. 'Semilla'
  6. 'Fecha de inicio'
  7. 'Fecha de la última acción'
  8. 'Dirección IP'
  9. 'URL de referencia'
  10. 'Indica, por favor, su lugar de residencia habitual'
  11. 'Indique provincia'
  12. 'Indique País de residencia'
  13. '¿Desde dónde viajó para venir a Valencia?'
  14. '¿Qué medios de transporte utilizó para llegar a Valencia? [Avión]'
  15. '¿Qué medios de transporte utilizó para llegar a Valencia? [Tren]'
  16. '¿Qué medios de transporte utilizó para llegar a Valencia? [Autobus]'
  17. '¿Qué medios de transporte utilizó para llegar a Valencia? [Vehículo propio/alquilado]'
  18. '¿Qué medios de transporte utilizó para llegar a Valencia? [Barco]'
  19. '¿Qué medios de transporte utilizó para llegar a Valencia? [Otro]'
  20. '¿Vino a Valencia con algún acompañante que no participó en el Congreso?'
  

In [55]:
#  Crear backup de datos originales

df_backup = df.copy()
print('Backup creado')

# Verificar el backup
print(f'Backup dimensions: {df_backup.shape}')


Backup creado
Backup dimensions: (47, 137)


In [56]:
[c for c in df.columns if c.startswith("Temporización de la pregunta:")]

['Temporización de la pregunta: Q01',
 'Temporización de la pregunta: Q02',
 'Temporización de la pregunta: Q03',
 'Temporización de la pregunta: Q04',
 'Temporización de la pregunta: Q05',
 'Temporización de la pregunta: Q06',
 'Temporización de la pregunta: Q07',
 'Temporización de la pregunta: Q08',
 'Temporización de la pregunta: Q09',
 'Temporización de la pregunta: Q10',
 'Temporización de la pregunta: Q11',
 'Temporización de la pregunta: Q12',
 'Temporización de la pregunta: Q13',
 'Temporización de la pregunta: Q14',
 'Temporización de la pregunta: Q15',
 'Temporización de la pregunta: Q16',
 'Temporización de la pregunta: Q17',
 'Temporización de la pregunta: Q18',
 'Temporización de la pregunta: Q19',
 'Temporización de la pregunta: Q20',
 'Temporización de la pregunta: Q21',
 'Temporización de la pregunta: Q22',
 'Temporización de la pregunta: Q23',
 'Temporización de la pregunta: Q24',
 'Temporización de la pregunta: Q25',
 'Temporización de la pregunta: Q00026']

In [57]:
cols_to_drop = {
    'Dirección IP',
    'Semilla',
    'URL de referencia',
    'Temporización de la pregunta: Q01',
    'Temporización de la pregunta: Q01',
    'Temporización de la pregunta: Q02',
    'Temporización de la pregunta: Q03',
    'Temporización de la pregunta: Q04',
    'Temporización de la pregunta: Q05',
    'Temporización de la pregunta: Q06',
    'Temporización de la pregunta: Q07',
    'Temporización de la pregunta: Q08',
    'Temporización de la pregunta: Q09',
    'Temporización de la pregunta: Q10',
    'Temporización de la pregunta: Q11',
    'Temporización de la pregunta: Q12',
    'Temporización de la pregunta: Q13',
    'Temporización de la pregunta: Q14',
    'Temporización de la pregunta: Q15',
    'Temporización de la pregunta: Q16',
    'Temporización de la pregunta: Q17',
    'Temporización de la pregunta: Q18',
    'Temporización de la pregunta: Q19',
    'Temporización de la pregunta: Q20',
    'Temporización de la pregunta: Q21',
    'Temporización de la pregunta: Q22',
    'Temporización de la pregunta: Q23',
    'Temporización de la pregunta: Q24',
    'Temporización de la pregunta: Q25',
    'Temporización de la pregunta: Q00026'

}

df = df.drop(columns=cols_to_drop)
print(f"Columnas eliminadas ({len(cols_to_drop)}):")
print(cols_to_drop)


Columnas eliminadas (29):
{'Temporización de la pregunta: Q05', 'Temporización de la pregunta: Q13', 'Temporización de la pregunta: Q03', 'Temporización de la pregunta: Q02', 'Temporización de la pregunta: Q10', 'Temporización de la pregunta: Q07', 'Temporización de la pregunta: Q16', 'Temporización de la pregunta: Q06', 'Temporización de la pregunta: Q20', 'Temporización de la pregunta: Q17', 'Temporización de la pregunta: Q15', 'Temporización de la pregunta: Q04', 'Temporización de la pregunta: Q01', 'Temporización de la pregunta: Q25', 'Temporización de la pregunta: Q09', 'Temporización de la pregunta: Q08', 'Dirección IP', 'Temporización de la pregunta: Q18', 'Temporización de la pregunta: Q19', 'Semilla', 'URL de referencia', 'Temporización de la pregunta: Q00026', 'Temporización de la pregunta: Q23', 'Temporización de la pregunta: Q14', 'Temporización de la pregunta: Q24', 'Temporización de la pregunta: Q21', 'Temporización de la pregunta: Q12', 'Temporización de la pregunta: Q22

In [58]:
#  Renombrar columnas

renombres = {
    # Perfil
    'Indica, por favor, su lugar de residencia habitual': 'residencia',
    'Indique provincia': 'provincia',
    'Indique País de residencia': 'pais',
    '¿Desde dónde viajó para venir a Valencia?': 'origen_viaje',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Avión]': 'transporte_llegada_avion',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Tren]': 'transporte_llegada_tren',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Autobus]': 'transporte_llegada_bus',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Vehículo propio/alquilado]': 'transporte_llegada_coche',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Barco]': 'transporte_llegada_barco',
    '¿Qué medios de transporte utilizó para llegar a Valencia? [Otro]': 'transporte_llegada_otro',
    '¿Vino a Valencia con algún acompañante que no participó en el Congreso?': 'acompanante',
    '¿Cuántas personas le acompañaron que no participaron el el Congreso?': 'num_acompanantes',
    '¿En qué rango de edad se encuentra?': 'edad_rango',
    '¿Cuál es su género?': 'genero',
    '¿Cuál es su género? [Otro]': 'genero_otro',
    'Por favor, seleccione la categoría que mejor describe su profesión principal.': 'profesion',
    'Por favor, seleccione la categoría que mejor describe su profesión principal. [Otro]': 'profesion_otro',
    '¿Qué rol desempeña principalmente en su actividad profesional?': 'rol_profesional',
    '¿Qué rol desempeña principalmente en su actividad profesional? [Otro]': 'rol_profesional_otro',
    '¿Cuál fue su rol en el Congreso?': 'rol_congreso',
    '¿Cuál fue su rol en el Congreso? [Otro]': 'rol_congreso_otro',
    '¿Ha asistido anteriormente a Congresos en Valencia?': 'asistencia_previa',
    
    # Alojamiento
    '¿Su estancia en Valencia se limitó a los días del congreso o se amplió más allá?': 'duracion_estancia',
    '¿Cuántas noches, en total, se quedó en Valencia?': 'noches_valencia',
    '¿Qué tipo de alojamiento utilizó durante su estancia (en caso de haber pernoctado fuera de su domicilio habitual)?': 'alojamiento',
    '¿Qué tipo de alojamiento utilizó durante su estancia (en caso de haber pernoctado fuera de su domicilio habitual)? [Otro]': 'alojamiento_otro',
    '¿Podría indicarnos dónde se hospedó durante su estancia en Valencia?': 'ubicacion_alojamiento',
    
    # Desplazamientos
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Taxi o VTC (Cabify, Uber)][Para asistir al congreso]': 'uso_taxi_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Taxi o VTC (Cabify, Uber)][Para actividades fuera del congreso]': 'uso_taxi_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Autobus][Para asistir al congreso]': 'uso_bus_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Autobus][Para actividades fuera del congreso]': 'uso_bus_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Metro/Tranvía][Para asistir al congreso]': 'uso_metro_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Metro/Tranvía][Para actividades fuera del congreso]': 'uso_metro_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Coche particular o de alquiler][Para asistir al congreso]': 'uso_coche_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Coche particular o de alquiler][Para actividades fuera del congreso]': 'uso_coche_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Bicicleta o patinete][Para asistir al congreso]': 'uso_bici_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Bicicleta o patinete][Para actividades fuera del congreso]': 'uso_bici_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Lanzadera][Para asistir al congreso]': 'uso_lanzadera_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [Lanzadera][Para actividades fuera del congreso]': 'uso_lanzadera_ocio',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [A pie][Para asistir al congreso]': 'uso_pie_congreso',
    '¿Cuántas veces utilizó cada uno de los siguientes medios de transporte durante su estancia en Valencia para: 1. Asistir al congreso 2. Realizar actividades fuera del Congreso (ocio, turismo, cenas, etc) [A pie][Para actividades fuera del congreso]': 'uso_pie_ocio',
    
    # Alimentación
    '¿Cuántas veces comió fuera del congreso? [Carne roja (Vacuno, cordero, cerdo)][Restaurante local]': 'carne_roja_restaurante',
    '¿Cuántas veces comió fuera del congreso? [Carne roja (Vacuno, cordero, cerdo)][Cadena de comida rápida]': 'carne_roja_fastfood',
    '¿Cuántas veces comió fuera del congreso? [Carne roja (Vacuno, cordero, cerdo)][Pedidos a domicilio]': 'carne_roja_domicilio',
    '¿Cuántas veces comió fuera del congreso? [Carne roja (Vacuno, cordero, cerdo)][Cociné en el alojamiento]': 'carne_roja_casera',
    '¿Cuántas veces comió fuera del congreso? [Carne de ave o pescado][Restaurante local]': 'avepescado_restaurante',
    '¿Cuántas veces comió fuera del congreso? [Carne de ave o pescado][Cadena de comida rápida]': 'avepescado_fastfood',
    '¿Cuántas veces comió fuera del congreso? [Carne de ave o pescado][Pedidos a domicilio]': 'avepescado_domicilio',
    '¿Cuántas veces comió fuera del congreso? [Carne de ave o pescado][Cociné en el alojamiento]': 'avepescado_casera',
    '¿Cuántas veces comió fuera del congreso? [Marisco (Bogavante, Langostinos, etc)][Restaurante local]': 'marisco_restaurante',
    '¿Cuántas veces comió fuera del congreso? [Marisco (Bogavante, Langostinos, etc)][Cadena de comida rápida]': 'marisco_fastfood',
    '¿Cuántas veces comió fuera del congreso? [Marisco (Bogavante, Langostinos, etc)][Pedidos a domicilio]': 'marisco_domicilio',
    '¿Cuántas veces comió fuera del congreso? [Marisco (Bogavante, Langostinos, etc)][Cociné en el alojamiento]': 'marisco_casera',

    # Turismo
    '¿Realizó visitas turísticas durante su estancia en Valencia?': 'visitas_turisticas',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Centro Histórico][Eje 1]': 'turismo_centrohistorico_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Centro Histórico][Eje 2]': 'turismo_centrohistorico_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Playa][Eje 1]': 'turismo_playa_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Playa][Eje 2]': 'turismo_playa_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Ciudad de las Artes y las Ciencias][Eje 1]': 'turismo_cac_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Ciudad de las Artes y las Ciencias][Eje 2]': 'turismo_cac_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Bioparc][Eje 1]': 'turismo_bioparc_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Bioparc][Eje 2]': 'turismo_bioparc_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [La Catedral][Eje 1]': 'turismo_catedral_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [La Catedral][Eje 2]': 'turismo_catedral_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [La Lonja][Eje 1]': 'turismo_lonja_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [La Lonja][Eje 2]': 'turismo_lonja_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Albufera][Eje 1]': 'turismo_albufera_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Albufera][Eje 2]': 'turismo_albufera_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [L´Almoina][Eje 1]': 'turismo_almoina_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [L´Almoina][Eje 2]': 'turismo_almoina_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Museo de Cerámica][Eje 1]': 'turismo_museoceramica_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Museo de Cerámica][Eje 2]': 'turismo_museoceramica_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Mercado Central][Eje 1]': 'turismo_mercadocentral_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Mercado Central][Eje 2]': 'turismo_mercadocentral_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Jardín Botánico][Eje 1]': 'turismo_jardinbotanico_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Jardín Botánico][Eje 2]': 'turismo_jardinbotanico_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Parque Turia][Eje 1]': 'turismo_turia_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Parque Turia][Eje 2]': 'turismo_turia_origen',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Otro][Eje 1]': 'turismo_otro_transporte',
    'Indique si visitó alguno de los siguientes lugares durante su estancia en Valencia, así como el medio de transporte que utilizó y el lugar desde el que partió. [Otro][Eje 2]': 'turismo_otro_origen',
    'Por favor, especifique el lugar que has visitado que no aparece en la lista.': 'turismo_otro_especificar',

    # Compras
    '¿Realizó compras durante su estancia en Valencia?': 'compras_realizo',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Productos textiles (ropa, pañuelos, etc.)][Eje 1]': 'compras_textiles_cantidad',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Productos textiles (ropa, pañuelos, etc.)][Eje 2]': 'compras_textiles_origen',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Artesanía (cerámica, madera, artículos de piel, etc.)][Eje 1]': 'compras_artesania_cantidad',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Artesanía (cerámica, madera, artículos de piel, etc.)][Eje 2]': 'compras_artesania_origen',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Productos alimenticios locales (vino, aceite, dulces, etc.)][Eje 1]': 'compras_alimentacion_cantidad',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Productos alimenticios locales (vino, aceite, dulces, etc.)][Eje 2]': 'compras_alimentacion_origen',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Souvenirs de producción masiva (imanes, llaveros, réplicas, etc.)][Eje 1]': 'compras_souvenirs_cantidad',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Souvenirs de producción masiva (imanes, llaveros, réplicas, etc.)][Eje 2]': 'compras_souvenirs_origen',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Otro][Eje 1]': 'compras_otro_cantidad',
    'Por favor, para cada tipo de producto adquirido, indique la cantidad y, si lo conoce, su lugar de origen. [Otro][Eje 2]': 'compras_otro_origen',
    'Por favor especifique el tipo de producto adquirido.': 'compras_otro_especificar',

    # Sugerencias
    '¿Hay algo más que nos quiera comentar? Puede utilizar este espacio para compartir sugerencias, incidencias o cualquier otra observación relacionada con su experiencia.': 'sugerencias'
}

print('Renombrando columnas...')

# Comprobar columnas que existen para renombrar
print('Columnas disponibles para renombrar:')
for col_original in renombres.keys():
    if col_original in df.columns:
        print(f"   ✅ '{col_original}' → '{renombres[col_original]}'")
    else:
        print(f"   ❌ '{col_original}' no encontrada")


# Aplicar renombres solo si las columnas existen
renombres_aplicados = {}
for col_original, col_nueva in renombres.items():
    if col_original in df.columns:
        renombres_aplicados[col_original] = col_nueva

df = df.rename(columns=renombres_aplicados)
print(f"Columnas renombradas: {len(renombres_aplicados)}")



Renombrando columnas...
Columnas disponibles para renombrar:
   ✅ 'Indica, por favor, su lugar de residencia habitual' → 'residencia'
   ✅ 'Indique provincia' → 'provincia'
   ✅ 'Indique País de residencia' → 'pais'
   ✅ '¿Desde dónde viajó para venir a Valencia?' → 'origen_viaje'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Avión]' → 'transporte_llegada_avion'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Tren]' → 'transporte_llegada_tren'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Autobus]' → 'transporte_llegada_bus'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Vehículo propio/alquilado]' → 'transporte_llegada_coche'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Barco]' → 'transporte_llegada_barco'
   ✅ '¿Qué medios de transporte utilizó para llegar a Valencia? [Otro]' → 'transporte_llegada_otro'
   ✅ '¿Vino a Valencia con algún acompañante que no participó en el Congreso?' → 'acompana

In [59]:
# Ver todas las columnas
print('Columnas renombradas:')
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. '{col}'")

Columnas renombradas:
  1. 'ID de respuesta'
  2. 'Fecha de envío'
  3. 'Última página'
  4. 'Lenguaje inicial'
  5. 'Fecha de inicio'
  6. 'Fecha de la última acción'
  7. 'residencia'
  8. 'provincia'
  9. 'pais'
  10. 'origen_viaje'
  11. 'transporte_llegada_avion'
  12. 'transporte_llegada_tren'
  13. 'transporte_llegada_bus'
  14. 'transporte_llegada_coche'
  15. 'transporte_llegada_barco'
  16. 'transporte_llegada_otro'
  17. 'acompanante'
  18. 'num_acompanantes'
  19. 'edad_rango'
  20. 'genero'
  21. 'genero_otro'
  22. 'profesion'
  23. 'profesion_otro'
  24. 'rol_profesional'
  25. 'rol_profesional_otro'
  26. 'rol_congreso'
  27. 'rol_congreso_otro'
  28. 'asistencia_previa'
  29. 'duracion_estancia'
  30. 'noches_valencia'
  31. 'alojamiento'
  32. 'alojamiento_otro'
  33. 'ubicacion_alojamiento'
  34. 'uso_taxi_congreso'
  35. 'uso_taxi_ocio'
  36. 'uso_bus_congreso'
  37. 'uso_bus_ocio'
  38. 'uso_metro_congreso'
  39. 'uso_metro_ocio'
  40. 'uso_coche_congreso'
  41. 'u

In [79]:
# Explorar la columna 'Última página'

col_name = "Última página"

if col_name in df.columns:
    print(f"Columna '{col_name}' encontrada")
    print(f"\nValores únicos en '{col_name}':")
    valores_ultima_pagina = df[col_name].value_counts().sort_index()
    print(valores_ultima_pagina)
    
else:
    print(f"Columna '{col_name}' NO encontrada")
    

# Eliminar filas donde 'Última página' ! = 7

filas_inicial = len(df)
print("Eliminando filas donde 'Última página' ! = 7...")
print(f"Filas antes de filtrar: {filas_inicial}")

# Aplicar filtro si existe la columna
if col_name in df.columns:
    # Contar cuántas filas tienen Última página = 0
    filas_cero = len(df[df[col_name] == 0])
    print(f"Filas con '{col_name}' = 0: {filas_cero}")
    
    # Filtrar el dataframe
    df = df[df[col_name] == 7]
    
    filas_eliminadas = filas_inicial - len(df)
    print(f"Filas eliminadas: {filas_eliminadas}")
    print(f"Filas restantes: {len(df)}")
else:
    print(f"No se puede aplicar filtro. Columna '{col_name}' no existe.")



Columna 'Última página' encontrada

Valores únicos en 'Última página':
Última página
1     2
2     2
7    33
Name: count, dtype: int64
Eliminando filas donde 'Última página' ! = 7...
Filas antes de filtrar: 37
Filas con 'Última página' = 0: 0
Filas eliminadas: 4
Filas restantes: 33


# Data type

In [80]:
yes_values = {'Sí'}
no_values = {'No'}

# Detectar columnas binarias
binary_cols = []
for c in df.columns:
    uniques = set(df[c].dropna().unique())
    if uniques and uniques.issubset(yes_values.union(no_values)):
        binary_cols.append(c)

print(f"Columnas binarias detectadas ({len(binary_cols)}):")
print(binary_cols)


Columnas binarias detectadas (0):
[]


In [62]:
for c in binary_cols:
    print(f"\nColumna: {c}")
    print(df[c].value_counts(dropna=False))


Columna: transporte_llegada_avion
transporte_llegada_avion
Sí    23
No    14
Name: count, dtype: int64

Columna: transporte_llegada_tren
transporte_llegada_tren
No    28
Sí     9
Name: count, dtype: int64

Columna: transporte_llegada_bus
transporte_llegada_bus
No    30
Sí     7
Name: count, dtype: int64

Columna: transporte_llegada_coche
transporte_llegada_coche
No    30
Sí     7
Name: count, dtype: int64

Columna: transporte_llegada_barco
transporte_llegada_barco
No    37
Name: count, dtype: int64

Columna: acompanante
acompanante
No    28
Sí     9
Name: count, dtype: int64

Columna: asistencia_previa
asistencia_previa
No    23
Sí    14
Name: count, dtype: int64

Columna: visitas_turisticas
visitas_turisticas
Sí     23
No     10
NaN     4
Name: count, dtype: int64

Columna: compras_realizo
compras_realizo
Sí     18
No     15
NaN     4
Name: count, dtype: int64


In [63]:
import unidecode

yes_values = {'si'}
no_values  = {'no'}

def convertir_binario(valor):
    if pd.isna(valor):
        return np.nan
    val = unidecode.unidecode(str(valor).strip().lower())  # quita tildes y espacios
    if val in yes_values:
        return 1
    elif val in no_values:
        return 0
    else:
        return np.nan

# Aplicar a todas las columnas binarias detectadas
for c in binary_cols:
    df[c] = df[c].apply(convertir_binario).astype('Int8')

In [41]:
for c in binary_cols:
    print(f"\nColumna: {c} (DESPUÉS de convertir)")
    print(df[c].value_counts(dropna=False))



Columna: transporte_llegada_avion (DESPUÉS de convertir)
transporte_llegada_avion
1    23
0    14
Name: count, dtype: Int64

Columna: transporte_llegada_tren (DESPUÉS de convertir)
transporte_llegada_tren
0    28
1     9
Name: count, dtype: Int64

Columna: transporte_llegada_bus (DESPUÉS de convertir)
transporte_llegada_bus
0    30
1     7
Name: count, dtype: Int64

Columna: transporte_llegada_coche (DESPUÉS de convertir)
transporte_llegada_coche
0    30
1     7
Name: count, dtype: Int64

Columna: transporte_llegada_barco (DESPUÉS de convertir)
transporte_llegada_barco
0    37
Name: count, dtype: Int64

Columna: acompanante (DESPUÉS de convertir)
acompanante
0    28
1     9
Name: count, dtype: Int64

Columna: asistencia_previa (DESPUÉS de convertir)
asistencia_previa
0    23
1    14
Name: count, dtype: Int64

Columna: visitas_turisticas (DESPUÉS de convertir)
visitas_turisticas
1       23
0       10
<NA>     4
Name: count, dtype: Int64

Columna: compras_realizo (DESPUÉS de convertir)


In [64]:
object_cols = df.select_dtypes(include='object').columns
print(object_cols)

Index(['Fecha de envío', 'Lenguaje inicial', 'Fecha de inicio',
       'Fecha de la última acción', 'residencia', 'provincia', 'pais',
       'origen_viaje', 'transporte_llegada_otro', 'edad_rango', 'genero',
       'profesion', 'rol_profesional', 'rol_profesional_otro', 'rol_congreso',
       'duracion_estancia', 'noches_valencia', 'alojamiento',
       'ubicacion_alojamiento', 'turismo_centrohistorico_transporte',
       'turismo_centrohistorico_origen', 'turismo_playa_transporte',
       'turismo_playa_origen', 'turismo_cac_transporte', 'turismo_cac_origen',
       'turismo_bioparc_transporte', 'turismo_bioparc_origen',
       'turismo_catedral_transporte', 'turismo_catedral_origen',
       'turismo_lonja_transporte', 'turismo_lonja_origen',
       'turismo_albufera_transporte', 'turismo_albufera_origen',
       'turismo_almoina_transporte', 'turismo_almoina_origen',
       'turismo_museoceramica_transporte', 'turismo_museoceramica_origen',
       'turismo_mercadocentral_transporte'

In [65]:
# Fechas
date_cols = ['Fecha de envío', 'Fecha de inicio', 'Fecha de la última acción']
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

In [66]:
object_cols = df.select_dtypes(include='object').columns
print(object_cols)

Index(['Lenguaje inicial', 'residencia', 'provincia', 'pais', 'origen_viaje',
       'transporte_llegada_otro', 'edad_rango', 'genero', 'profesion',
       'rol_profesional', 'rol_profesional_otro', 'rol_congreso',
       'duracion_estancia', 'noches_valencia', 'alojamiento',
       'ubicacion_alojamiento', 'turismo_centrohistorico_transporte',
       'turismo_centrohistorico_origen', 'turismo_playa_transporte',
       'turismo_playa_origen', 'turismo_cac_transporte', 'turismo_cac_origen',
       'turismo_bioparc_transporte', 'turismo_bioparc_origen',
       'turismo_catedral_transporte', 'turismo_catedral_origen',
       'turismo_lonja_transporte', 'turismo_lonja_origen',
       'turismo_albufera_transporte', 'turismo_albufera_origen',
       'turismo_almoina_transporte', 'turismo_almoina_origen',
       'turismo_museoceramica_transporte', 'turismo_museoceramica_origen',
       'turismo_mercadocentral_transporte', 'turismo_mercadocentral_origen',
       'turismo_jardinbotanico_transpo

In [68]:
# Función genérica para mapear y convertir
def map_numeric_column(df, col, mapping):
    print(f"\nColumna: {col} (ANTES de convertir)")
    print(df[col].value_counts(dropna=False))

    # Limpiar valores
    df[col] = df[col].astype(str).str.strip()           # quitar espacios
    df[col] = df[col].apply(lambda x: unidecode.unidecode(x.lower()))  # quitar tildes y pasar a minúscula

    # Limpiar también las claves del mapping
    mapping_clean = {unidecode.unidecode(k.lower()): v for k, v in mapping.items()}

    # Mapear y reemplazar NaN por 0
    df[col] = df[col].map(mapping_clean).fillna(0).astype(int)

    print(f"\nColumna: {col} (DESPUÉS de convertir)")
    print(df[col].value_counts(dropna=False))

# Mapear noches_valencia
noches_map = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    'más de 10': 10
}
map_numeric_column(df, 'noches_valencia', noches_map)

# Mapear columnas de shopping
shopping_map = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    'más de 5': 5
}
shopping_cols = [
    "compras_textiles_cantidad",
    "compras_artesania_cantidad",
    "compras_alimentacion_cantidad",
    "compras_souvenirs_cantidad"
]

for c in shopping_cols:
    map_numeric_column(df, c, shopping_map)



Columna: noches_valencia (ANTES de convertir)
noches_valencia
4            9
2            6
5            6
3            5
más de 10    4
7            3
NaN          2
1            1
0            1
Name: count, dtype: int64

Columna: noches_valencia (DESPUÉS de convertir)
noches_valencia
4     9
2     6
5     6
3     5
10    4
7     3
0     3
1     1
Name: count, dtype: int64

Columna: compras_textiles_cantidad (ANTES de convertir)
compras_textiles_cantidad
NaN         31
1            4
2            1
más de 5     1
Name: count, dtype: int64

Columna: compras_textiles_cantidad (DESPUÉS de convertir)
compras_textiles_cantidad
0    31
1     4
2     1
5     1
Name: count, dtype: int64

Columna: compras_artesania_cantidad (ANTES de convertir)
compras_artesania_cantidad
NaN    35
2.0     1
1.0     1
Name: count, dtype: int64

Columna: compras_artesania_cantidad (DESPUÉS de convertir)
compras_artesania_cantidad
0    37
Name: count, dtype: int64

Columna: compras_alimentacion_cantidad (ANTES 

In [69]:
object_cols = df.select_dtypes(include='object').columns
print(object_cols)

Index(['Lenguaje inicial', 'residencia', 'provincia', 'pais', 'origen_viaje',
       'transporte_llegada_otro', 'edad_rango', 'genero', 'profesion',
       'rol_profesional', 'rol_profesional_otro', 'rol_congreso',
       'duracion_estancia', 'alojamiento', 'ubicacion_alojamiento',
       'turismo_centrohistorico_transporte', 'turismo_centrohistorico_origen',
       'turismo_playa_transporte', 'turismo_playa_origen',
       'turismo_cac_transporte', 'turismo_cac_origen',
       'turismo_bioparc_transporte', 'turismo_bioparc_origen',
       'turismo_catedral_transporte', 'turismo_catedral_origen',
       'turismo_lonja_transporte', 'turismo_lonja_origen',
       'turismo_albufera_transporte', 'turismo_albufera_origen',
       'turismo_almoina_transporte', 'turismo_almoina_origen',
       'turismo_museoceramica_transporte', 'turismo_museoceramica_origen',
       'turismo_mercadocentral_transporte', 'turismo_mercadocentral_origen',
       'turismo_jardinbotanico_transporte', 'turismo_jard

In [71]:
num_cols = df.select_dtypes(include='number').columns
print(num_cols)


Index(['ID de respuesta', 'Última página', 'transporte_llegada_avion',
       'transporte_llegada_tren', 'transporte_llegada_bus',
       'transporte_llegada_coche', 'transporte_llegada_barco', 'acompanante',
       'num_acompanantes', 'genero_otro', 'profesion_otro',
       'rol_congreso_otro', 'asistencia_previa', 'noches_valencia',
       'alojamiento_otro', 'uso_taxi_congreso', 'uso_taxi_ocio',
       'uso_bus_congreso', 'uso_bus_ocio', 'uso_metro_congreso',
       'uso_metro_ocio', 'uso_coche_congreso', 'uso_coche_ocio',
       'uso_bici_congreso', 'uso_bici_ocio', 'uso_lanzadera_congreso',
       'uso_lanzadera_ocio', 'uso_pie_congreso', 'uso_pie_ocio',
       'carne_roja_restaurante', 'carne_roja_fastfood', 'carne_roja_domicilio',
       'carne_roja_casera', 'avepescado_restaurante', 'avepescado_fastfood',
       'avepescado_domicilio', 'avepescado_casera', 'marisco_restaurante',
       'marisco_fastfood', 'marisco_domicilio', 'marisco_casera',
       'visitas_turisticas', 'comp

In [74]:
text_cols = ['genero_otro', 'profesion_otro', 'rol_congreso_otro']

for c in text_cols:
    df[c] = df[c].astype(str)

In [75]:
num_cols = df.select_dtypes(include='number').columns
print(num_cols)

Index(['ID de respuesta', 'Última página', 'transporte_llegada_avion',
       'transporte_llegada_tren', 'transporte_llegada_bus',
       'transporte_llegada_coche', 'transporte_llegada_barco', 'acompanante',
       'num_acompanantes', 'asistencia_previa', 'noches_valencia',
       'alojamiento_otro', 'uso_taxi_congreso', 'uso_taxi_ocio',
       'uso_bus_congreso', 'uso_bus_ocio', 'uso_metro_congreso',
       'uso_metro_ocio', 'uso_coche_congreso', 'uso_coche_ocio',
       'uso_bici_congreso', 'uso_bici_ocio', 'uso_lanzadera_congreso',
       'uso_lanzadera_ocio', 'uso_pie_congreso', 'uso_pie_ocio',
       'carne_roja_restaurante', 'carne_roja_fastfood', 'carne_roja_domicilio',
       'carne_roja_casera', 'avepescado_restaurante', 'avepescado_fastfood',
       'avepescado_domicilio', 'avepescado_casera', 'marisco_restaurante',
       'marisco_fastfood', 'marisco_domicilio', 'marisco_casera',
       'visitas_turisticas', 'compras_realizo', 'compras_textiles_cantidad',
       'compras_ar

In [77]:
df[num_cols].describe()

,ID de respuesta,Última página,transporte_llegada_avion,transporte_llegada_tren,transporte_llegada_bus,transporte_llegada_coche,transporte_llegada_barco,acompanante,num_acompanantes,asistencia_previa,...,compras_souvenirs_cantidad,compras_otro_cantidad,Tiempo total,Temporización del grupo: Perfil del Congresista,Temporización del grupo: Alojamiento,Temporización del grupo: Desplazamientos,Temporización del grupo: Alimentación,Temporización del grupo: Actividades,Temporización del grupo: Compras,Temporización del grupo: Sugerencias
count,37.000000,37.000000,37.0,37.0,37.0,37.0,37.0,37.0,9.000000,37.0,...,37.000000,4.0,37.000000,37.000000,35.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,24.243243,6.405405,0.621622,0.243243,0.189189,0.189189,0.0,0.243243,1.444444,0.378378,...,0.675676,2.0,678.884595,241.663784,85.360286,80.292727,139.483939,85.784545,38.184545,55.937879
std,13.200180,1.739404,0.491672,0.434959,0.397061,0.397061,0.0,0.434959,0.726483,0.491672,...,1.225970,2.0,571.290786,280.712601,59.297687,92.645619,375.369463,142.929598,50.900696,163.770921
min,2.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.000000,1.0,140.430000,55.190000,17.130000,13.490000,10.550000,5.120000,2.600000,2.470000
25%,14.000000,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.000000,1.0,339.730000,99.910000,45.040000,36.710000,29.500000,13.380000,8.230000,4.610000
50%,25.000000,7.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.000000,1.0,449.720000,163.560000,70.480000,50.050000,48.490000,55.510000,25.090000,7.520000
75%,34.000000,7.000000,1.0,0.0,0.0,0.0,0.0,0.0,2.000000,1.0,...,1.000000,2.0,791.280000,275.710000,97.160000,82.050000,67.900000,82.850000,43.020000,41.110000
max,47.000000,7.000000,1.0,1.0,1.0,1.0,0.0,1.0,3.000000,1.0,...,5.000000,5.0,2604.090000,1430.540000,319.770000,462.840000,2086.730000,664.940000,260.490000,931.010000


In [81]:
#  Reporte final y exploración

print('REPORTE FINAL:')
print('='*50)
print(f'Dimensiones originales: {df_backup.shape}')
print(f'Dimensiones finales: {df.shape}')
print(f'Filas eliminadas: {filas_eliminadas}')
print(f'Columnas renombradas: {len(renombres_aplicados)}')

# Información general del dataset procesado
print('\nInformación del dataset procesado:')
df.info()

# Resumen del número de columnas por tipo de dato
print('\nNúmero de columnas por tipo de dato:')
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f'   {dtype}: {count} columnas')

# Verificar valores nulos
print('\nValores nulos por columna:')
valores_nulos = df.isnull().sum()
for col, nulos in valores_nulos.items():
    if nulos > 0:
        porcentaje = (nulos / len(df)) * 100
        print(f'   {col}: {nulos} ({porcentaje:.1f}%)')

if valores_nulos.sum() == 0:
    print('   ✅ No hay valores nulos')




REPORTE FINAL:
Dimensiones originales: (47, 137)
Dimensiones finales: (33, 108)
Filas eliminadas: 4
Columnas renombradas: 94

Información del dataset procesado:
<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 2 to 46
Columns: 108 entries, ID de respuesta to Temporización del grupo: Sugerencias
dtypes: Int8(9), datetime64[ns](3), float64(37), int64(7), object(52)
memory usage: 26.4+ KB

Número de columnas por tipo de dato:
   object: 52 columnas
   float64: 37 columnas
   Int8: 9 columnas
   int64: 7 columnas
   datetime64[ns]: 3 columnas

Valores nulos por columna:
   provincia: 24 (72.7%)
   pais: 9 (27.3%)
   transporte_llegada_otro: 29 (87.9%)
   num_acompanantes: 25 (75.8%)
   rol_profesional_otro: 32 (97.0%)
   alojamiento_otro: 33 (100.0%)
   uso_taxi_congreso: 24 (72.7%)
   uso_taxi_ocio: 25 (75.8%)
   uso_bus_congreso: 19 (57.6%)
   uso_bus_ocio: 21 (63.6%)
   uso_metro_congreso: 16 (48.5%)
   uso_metro_ocio: 24 (72.7%)
   uso_coche_congreso: 28 (84.8%)
   uso_coche_oc

In [ ]:
# Guardar en Excel
archivo_salida = '/Users/user/projects/projects/zentropyETL/data/encuesta_procesada.xlsx'

print(f'Guardando resultados en: {archivo_salida}')

with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
    # Datos procesados
    df.to_excel(writer, sheet_name='Datos_Procesados', index=False)
    
    # Datos originales (backup)
    df_backup.to_excel(writer, sheet_name='Datos_Originales', index=False)
    
    # Resumen del procesamiento
    resumen = pd.DataFrame({
        'Métrica': [
            'Filas originales', 
            'Filas procesadas', 
            'Filas eliminadas',
            'Columnas totales',
            'Columnas renombradas'
        ],
        'Valor': [
            len(df_backup), 
            len(df), 
            filas_eliminadas,
            len(df.columns),
            len(renombres_aplicados)
        ]
    })
    resumen.to_excel(writer, sheet_name='Resumen', index=False)

print('Archivo guardado exitosamente!')

# Mostrar resumen final
print('\nPROCESAMIENTO COMPLETADO!')
print(f'Dataset final: {df.shape[0]} filas × {df.shape[1]} columnas')
print(f'Archivo guardado: {archivo_salida}')

Guardando resultados en: encuesta_procesada.xlsx
Archivo guardado exitosamente!

PROCESAMIENTO COMPLETADO!
Dataset final: 35 filas × 104 columnas
Archivo guardado: encuesta_procesada.xlsx


# Agregación numericas y categoricas

In [ ]:
'''# Columnas sensibles
columnas_sensibles = ['Dirección IP', 'Fecha de envío', 'Fecha de inicio', 'Fecha de la última acción', 'origen_viaje', 'Semilla', 'URL de referencia', 'ubicacion_alojamiento']

# Eliminar columnas sensibles
df_sin_sensibles = df.drop(columns=[col for col in columnas_sensibles if col in df.columns])

# Ahora seleccionamos numéricas y categóricas sobre el DataFrame limpio
numericas = df_sin_sensibles.select_dtypes(include=['int64', 'float64'])
categoricas = df_sin_sensibles.select_dtypes(include=['object'])

# Confirmar
print("Columnas numéricas:", numericas.columns.tolist())
print("Columnas categóricas:", categoricas.columns.tolist())'''


Columnas numéricas: ['ID de respuesta', 'Última página', 'num_acompanantes', 'uso_taxi_congreso', 'uso_taxi_ocio', 'uso_bus_congreso', 'uso_bus_ocio', 'uso_metro_congreso', 'uso_metro_ocio', 'uso_coche_congreso', 'uso_coche_ocio', 'uso_bici_congreso', 'uso_bici_ocio', 'uso_pie_congreso', 'uso_pie_ocio', 'carne_roja_restaurante', 'carne_roja_fastfood', 'carne_roja_domicilio', 'carne_roja_casera', 'avepescado_restaurante', 'avepescado_fastfood', 'avepescado_domicilio', 'avepescado_casera', 'marisco_restaurante', 'marisco_domicilio', 'marisco_casera', 'compras_artesania_cantidad', 'compras_otro_cantidad', 'Tiempo total', 'Temporización del grupo: Perfil del Congresista', 'Temporización del grupo: Alojamiento', 'Temporización del grupo: Desplazamientos', 'Temporización del grupo: Alimentación', 'Temporización del grupo: Actividades', 'Temporización del grupo: Compras', 'Temporización del grupo: Sugerencias']
Columnas categóricas: ['Lenguaje inicial', 'residencia', 'provincia', 'pais', 'tra

In [ ]:
'''# Resumen variables numericas: media, mediana, std dev, min, max
resumen_numericas = numericas.agg(['mean', 'median', 'std', 'min', 'max']).T
print(resumen_numericas)
'''

                                                       mean  median  \
ID de respuesta                                   22.971429   24.00   
Última página                                      6.371429    7.00   
num_acompanantes                                   1.375000    1.00   
uso_taxi_congreso                                  2.666667    2.00   
uso_taxi_ocio                                      1.714286    2.00   
uso_bus_congreso                                   5.307692    6.00   
uso_bus_ocio                                       2.916667    2.00   
uso_metro_congreso                                 2.625000    2.00   
uso_metro_ocio                                     3.555556    2.00   
uso_coche_congreso                                 2.600000    3.00   
uso_coche_ocio                                     1.666667    2.00   
uso_bici_congreso                                  1.000000    1.00   
uso_bici_ocio                                      3.000000    4.00   
uso_pi

In [ ]:

'''print(categoricas.shape)
print(categoricas.isna().all().sum(), "columnas categóricas vacías")
'''

(35, 60)
0 columnas categóricas vacías


In [ ]:
'''for col in categoricas.columns:
    print("\n" + "="*60)
    print(f"Variable: {col}")
    print(categoricas[col].value_counts(dropna=False))'''


Variable: Lenguaje inicial
Lenguaje inicial
es    25
en    10
Name: count, dtype: int64

Variable: residencia
residencia
Extranjero    24
España        11
Name: count, dtype: int64

Variable: provincia
provincia
NaN                   24
Valencia/València      6
Castellón/Castelló     2
Murcia                 2
Barcelona              1
Name: count, dtype: int64

Variable: pais
pais
NaN                                                     11
Italy                                                    6
Slovenia                                                 4
Japan                                                    3
Mexico                                                   3
France                                                   2
Belgium                                                  1
Denmark                                                  1
Austria                                                  1
United Kingdom of Great Britain and Northern Ireland     1
Portugal                 

In [ ]:
# Crear un diccionario con todas las frecuencias
'''import json
frecuencias_dict = {}
for col in categoricas.columns:
    tabla = categoricas[col].value_counts(dropna=False)
    frecuencias_dict[col] = tabla.to_dict()  # convierte Series a diccionario

# Exportar a JSON
archivo_salida_json = '/Users/user/projects/projects/zentropyETL/data/frecuencias_categoricas.json'

with open(archivo_salida_json, "w", encoding="utf-8") as f:
    json.dump(frecuencias_dict, f, ensure_ascii=False, indent=4)'''

In [ ]:
# Guardar variables numéricas
'''archivo_salida = "/Users/user/projects/projects/zentropyETL/data/resumen_numericas.xlsx"

with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
    resumen_numericas.to_excel(writer, sheet_name='Resumen', index=False) '''
